# Using the Model

> This notebook explains how one can load a SysML v2 model and interact with the data

In [ ]:
from pathlib import Path

import pymbe.api as pm

In [ ]:
fixtures_folder = Path(pm.__file__).parent / "../../tests/fixtures/"

In [ ]:
model = pm.Model.load_from_file(fixtures_folder / "Kerbal.json")
# model = pm.Model.load_from_file(list(fixtures_folder.glob("*.json"))[6])

model

you can save the model back to a file

In [ ]:
model.save_to_file("New Kerbal.json")

and load it back...

In [ ]:
new_model = pm.Model.load_from_file("New Kerbal.json")

assert len(new_model.elements) == len(model.elements)

the model can own elements (`.ownedElement`) and relationships (`.ownedRelationship`), and an element can be retrieved by name

model.ownedElement[0].ownedElement["14a-Language Extensions"].ownedElement["User Defined Extensions"].ownedElement["ClassificationLevel"].data

In [ ]:
kerbal = model.ownedElement["Kerbal"]

assert kerbal.name == "Kerbal"

kerbal

this can be done sequentially through the chain of element ownership

In [ ]:
kerbal.ownedElement["Parts Library"].ownedElement["FL-T200 Fuel Tank"].ownedElement["Empty Mass"].ownedElement[0]["value"]

In [ ]:
assert kerbal.ownedElement["Parts Library"].ownedElement["FL-T200 Fuel Tank"].ownedElement["Empty Mass"].ownedElement[0].value == 0.125

all the model elements are stored under `model.elements` and can be retrieved by their SysML identifier

In [ ]:
a_return_parameter_membership = model.elements["20511137-d751-4baf-ac4b-994a8e65ad3d"]
a_return_parameter_membership

and their properties are resolved to the appropriate element

In [ ]:
a_return_parameter_membership.relatedElement[0].value

because of the nature of the data, there are multiple ways to refer to the same element, and the interactions allow for that

In [ ]:
assert a_return_parameter_membership.target.reverseReturnParameterMembership == a_return_parameter_membership.relatedElement[0]

In [ ]:
assert a_return_parameter_membership._id == a_return_parameter_membership._data["@id"]

In [ ]:
model.elements["0b6a7f54-f8bf-4a16-b5aa-a309321256c1"].source.throughSuperclassing.name == "Kerbal Rocket Part"

we can also explore all the relationships in a given element

a `through<Relationship Metatype>` is attached to the source, while a `reverse<Relationship Metatype>` is attached to the targer

In [ ]:
an_element = a_return_parameter_membership.relatedElement[1]

a_return_parameter_membership.target.relationships

we can also see all the relationships

In [ ]:
from pprint import pprint


for element_id, element in model.elements.items():
    name = element.qualifiedName or element_id
    relationships = element.relationships

    if not relationships:
        continue
    print("\n", name)
    pprint(relationships)